In [1]:
import os
import json
import pandas as pd 
import traceback 

In [7]:
import langchain

In [10]:

from langchain.chat_models import ChatOpenAI

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
KEY = os.getenv("OPENAI_API_KEY")

In [14]:
llm = ChatOpenAI(openai_api_key = KEY,model_name = "gpt-3.5-turbo",temperature = 0.5)

C:\Users\malah\AppData\Local\Temp\ipykernel_16980\1526675891.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key = KEY,model_name = "gpt-3.5-turbo",temperature = 0.5)


In [17]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [18]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [19]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [20]:
quiz_generation_prompt = PromptTemplate(
    input_variable = ["text","number","subject","tone","response_json"],
    template = TEMPLATE
)

In [21]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key = "quiz",verbose=True)

C:\Users\malah\AppData\Local\Temp\ipykernel_16980\3019009617.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key = "quiz",verbose=True)


In [22]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [23]:
quiz_evaluation_prompt = PromptTemplate(
    input_variable = ["subject","quiz"],
    template = TEMPLATE
)

In [24]:
review_chain = LLMChain(llm = llm,prompt = quiz_evaluation_prompt,output_key="review",verbose=True)

In [27]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain,review_chain],input_variables = ["text","number","subject","tone","response_json"],
output_variables = ["quiz","review"],verbose=True)

In [28]:
file_path = r"C:\Users\malah\mcqgen\data.txt"

In [29]:
with open(file_path,'r') as file:
    TEXT = file.read()

In [33]:
#our response is in dictionary format to change this to json we do,
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [34]:
NUMBER = 5
SUBJECT = "machine learning"
TONE = "simple"

In [37]:
#to track the token usage in langchain
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text":TEXT,
            "number":NUMBER,
            "subject":SUBJECT,
            "tone":TONE,
            "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approximately 

In [38]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1408
Prompt Tokens:946
Completion Tokens:462
Total Cost:0.002343


In [39]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine learning.',
 'number': 5,

In [45]:
quiz = response.get("quiz")

In [48]:
quiz = json.loads(quiz)

In [49]:
quiz_table_data = []
for key,value in quiz.items():
    mcq = value["mcq"]
    options = "|".join(
        [
            f"{option} : {option_value}"
            for option,option_value in value ["options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq,"choices":options,"correct":correct})

In [50]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'choices': 'a : A field of study in artificial intelligence concerned with developing and studying statistical algorithms that can learn from data|b : A field of study in natural language processing|c : A field of study in computer vision|d : A field of study in speech recognition',
  'correct': 'a'},
 {'MCQ': 'Which field does machine learning find applications in?',
  'choices': 'a : Agriculture|b : Astronomy|c : Zoology|d : Botany',
  'correct': 'a'},
 {'MCQ': 'What is the foundation of machine learning?',
  'choices': 'a : Data mining|b : Statistics and mathematical optimization|c : Exploratory data analysis|d : Natural language processing',
  'correct': 'b'},
 {'MCQ': 'What is the theoretical framework for describing machine learning?',
  'choices': 'a : Probably accurate correct (PAC) learning|b : Definitely accurate correct (DAC) learning|c : Approximately accurate correct (AAC) learning|d : Precisely accurate correct (PAC) learning',
  'c

In [52]:
quiz = pd.DataFrame(quiz_table_data)

In [53]:
quiz.to_csv("machinelearning.csv",index=False)